In [ ]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

import csv

import os

In [ ]:
def pegar_dados_operadora(coOperadora):
    chromedriver="C:\shared\chromedriver.exe"
    os.environ["webdriver.chrome.driver"] = chromedriver
    driver = webdriver.Chrome(chromedriver)

    url='http://www.ans.gov.br/ConsultaPlanosConsumidor/pages/ConsultaPlanos.xhtml?coOperadora=' + coOperadora
    driver.get(url)

    driver.implicitly_wait(3)

    # Pesquisa sem filtro
    driver.find_element_by_id('formConsulta:j_idt27').click()

    # Total de paginas
    pagina = 1
    paginas = int(driver.find_element_by_class_name('ui-paginator-current').text.split(' de ')[1])

    rows = []
    for i in range(paginas):
        html = BeautifulSoup(driver.page_source, 'html.parser')

        table_div = html.find('div' , {'id': 'formConsulta:tblPlanos' })
        table = table_div.find('table')

        for row in table.find_all('tr'):
            rows.append([val.text.replace('\n','') for val in row.find_all('td')])

        # Proxima página
        driver.find_element_by_class_name('ui-paginator-next').click()

        while True:
            atual = int(driver.find_element_by_class_name('ui-paginator-current').text.split(' de ')[0])

            if atual == paginas:
                break;

            if atual == pagina + 1:
                pagina = pagina + 1
                driver.implicitly_wait(1)
                break
            else:
                driver.implicitly_wait(3)

    # Gravar
    file = coOperadora + '.csv'
    with open(file, 'w', newline='', encoding='utf-16') as f:
        writer = csv.writer(f)
        writer.writerows(row for row in rows if row)

    # Fechar
    driver.quit()

In [ ]:
operadoras = ['368253', '311961', '005711', '367095', '416894', '006246', '366871', '313971', '000701', '406554', '333689']

for i, codigo in enumerate(operadoras):
    pegar_dados_operadora(codigo)